### Schach-Komponente ruft selber update-Funktion der Darstellungskomponente auf

Das Module `schach_1.py` unterscheidet sich von `schach_1.py`
durch eine kleine, aber konzeptuell wichtige Änderung. 
Es gib nun eine Funktion `update(event, **kwargs)`,
die im Moment nur ihre Argumente ausgibt, und
jeweils am Ende der Funktionen `move` und `new_game` aufgerufen wird.
Das 2. Argument enthält Information, die ein bequemes der Darstellung des Spielzustandes erlauben.

Später wird die Darstellungskompnente die Funktion
`update` durch ihre eigene update-Funktion mit der gleichen Signature 
überschreiben. 

Dadurch veranlasst die Spielkomponente das Update der graphischen Darstellung, und es genügt, das z.B. die Callback Funktion für das mouse-up Event die Funktion move der Spielkomponente aufruft.


```python
def update(event, **kwargs):
    print(f'event: {event}, kwargs: {kwargs}')

def move(src, target):
    ...
    changes = ((SPACE, *src), (char, *target))
    update('move', changes=changes)


def new_game():
    set_startpos()
    update('new_game', pieces=get_pieces())
```


```python
import schach_1 as schach
import schach_darstellung as D
...

schach.update = lambda event, **kwargs: D.update(fg, event, **kwargs)
```

In [1]:
import schach_1 as schach
import schach_darstellung as D
from ipycanvas import MultiCanvas
from time import sleep


layout = {'border': '1px solid black'}
mcanvas = MultiCanvas(2, width=200, height=200, layout=layout)
bg, fg = mcanvas

D.draw_chessboard(bg)

schach.update = lambda event, **kwargs: D.update(fg, event, **kwargs)
schach.new_game()
mcanvas

MultiCanvas(height=200, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_r…

In [8]:
moves = [('e2', 'e4'), ('e7', 'e5'), ('f1', 'c4'), ('f8', 'c5'),
         ('d1', 'h5'), ('g8', 'f6'), ('h5', 'f7')]

schach.new_game()
for move in moves:
    changes = schach.hmove(*move)
    sleep(1)